file:///home/tole/Downloads/Kron_Grund_Criticality.pdf
https://www.researchgate.net/publication/263333872_Society_as_a_Self-Organized_Critical_System
https://youtu.be/S83u_y3ZRYg


* relative criticality defined as proportion cells >= SOC / total nr of cells 

* relative criticality does not necessarily decrease after an avalance, but if it has decreased, then it is
* as a result of an avalance : an avalance is a necessary but not sufficient condition for decrease




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from graphics import *
import time

sns.set()

In [ ]:
GRID_SIDE = 49 ### use odd number for a well defined center int (N / 2)
OUTER_GRID_SIDE = GRID_SIDE + 2 
SOC = 3 # Self Organized Criticality - Critical Point #

GRAPHICS = False
DBG = False

np.random.seed(100)

full_grid = np.zeros((OUTER_GRID_SIDE,OUTER_GRID_SIDE)).reshape(OUTER_GRID_SIDE,-1).astype(int)

full_grid[1:-1,1:-1]  = np.random.randint(0,SOC+1,GRID_SIDE ** 2).reshape(GRID_SIDE,-1).astype(int)
#full_grid[1:-1,1:-1] = np.zeros((GRID_SIDE,GRID_SIDE))

def add_grain(grid,r,c):
    grid[r,c] += 1
    return grid

def inner_to_outer(r,c):
    return (r+1,c+1)

def reconfigure(full_grid,r,c):
    r,c = inner_to_outer(r,c)
    
    ### diagonals ###
    if full_grid[r,c] == 5:
        full_grid[r-1,c-1] += 1
    
    if full_grid[r,c] == 6:
        full_grid[r-1,c+1] += 1
        
    if full_grid[r,c] == 7:
        full_grid[r+1,c-1] += 1
        
    if full_grid[r,c] == 8 :
        full_grid[r+1,c+1] += 1
        
        
    full_grid[r+1,c] += 1
    full_grid[r-1,c] += 1
    full_grid[r,c+1] += 1
    full_grid[r,c-1] += 1

    full_grid[r,c] = 0
            
    return full_grid
    
def get_inner_grid():
    return full_grid[1:-1,1:-1]


inner = get_inner_grid()

#### GRAPHICS ####
graphics_factor = 30 # N pixels is a graphical unit (side)

if GRAPHICS:
    win = GraphWin(r'Self Organized Criticality - The Sandpile Model',
               GRID_SIDE * graphics_factor,GRID_SIDE * graphics_factor )

square_side = graphics_factor * GRID_SIDE // GRID_SIDE

colors = ['lightgrey','lightgreen','yellow','orange','red','crimson','darkred','navy','black']

def create_grid():
    rec_list = []
    r_list = []
    c_list = []
    
    for r in range(GRID_SIDE):
        for c in range(GRID_SIDE):
                
            rr = c * graphics_factor
            cc = r * graphics_factor

            rec = Rectangle(Point(rr,cc),
                Point((rr + square_side),(cc + square_side )))
            
            rec.setFill(colors[inner[r,c]])
        
            rec_list.append(rec)
            r_list.append(r)
            c_list.append(c)
    
    return rec_list,r_list,c_list

def redraw_grid(rec_list,r_list,c_list):
    
    for i in range(len(rec_list)):
        
        rec_list[i].undraw()

        rec_list[i].setFill(colors[ inner[ r_list[i],c_list[i] ] ])

        rec_list[i].draw(win)


rec_list,r_list,c_list = create_grid()

def add_marker(r,c):
    rr = c * graphics_factor + graphics_factor // 2
    cc = r * graphics_factor + graphics_factor // 2
    marker = Circle(Point(rr,cc),10)
    marker.setFill('black')
    marker.draw(win)
    return marker

#### GRAPHICS END ####


print ('start : \n',inner)
print()

ITERATIONS = int(1e6)

saturation = GRID_SIDE ** 2 * SOC

chain_reactions = np.zeros((ITERATIONS))
nr_grains = np.zeros(ITERATIONS)
rel_criticality = np.zeros(ITERATIONS)
rel_saturation = np.zeros(ITERATIONS)
max_pile = np.zeros(ITERATIONS)

#max_criticality = inner.shape[0] * inner.shape[1] * SOC
max_criticality = inner.shape[0] * inner.shape[1] # all cells >= SOC

status_factor = ITERATIONS // 10 if GRAPHICS else ITERATIONS // 1000

for i in range(ITERATIONS):
    
    #rel_criticality[i] = inner.sum() / max_criticality
    rel_criticality[i] = (inner >= SOC).sum() / max_criticality
    
    if GRAPHICS:
        redraw_grid(rec_list,r_list,c_list)
        time.sleep(0.2)
        
    
    if i % (ITERATIONS // status_factor) == 0:
        print ('\n+++ TIME : {} ITERATION : {} +++'.format(pd.Timestamp.now(),i))
    
    nr_grains[i] = get_inner_grid().sum()
    rel_saturation[i] = nr_grains[i] / saturation
    
    #### drop new grains at random ####
    #r0,c0 = np.random.randint(0,inner.shape[0],2)
    #inner = add_grain(inner,r0,c0) 
    
    #### drop new grains at center ####
    r0,c0 = GRID_SIDE // 2, GRID_SIDE // 2
    inner = add_grain(inner,GRID_SIDE // 2,GRID_SIDE // 2)
    
    if GRAPHICS:
        marker = add_marker(r0,c0)
        time.sleep(0.3)

    critical = True

    chain = 0
    while critical:
                
        if GRAPHICS:
            redraw_grid(rec_list,r_list,c_list)
            time.sleep(0.2)
        
       
        for r in range(GRID_SIDE):
            for c in range(GRID_SIDE):
                
                if inner[r,c] > SOC:  

                    full_grid = reconfigure(full_grid,r,c)
                    
                    if GRAPHICS:
                        redraw_grid(rec_list,r_list,c_list)
                        time.sleep(0.1)
                    
                    inner = get_inner_grid()
                    chain += 1
                    
                    if max_pile[i] < get_inner_grid().max():
                        max_pile[i] = get_inner_grid().max()
                    


        critical = (inner > SOC).any()
        
    chain_reactions[i] = chain
    
    ###redraw_grid(rec_list,r_list,c_list)

    if GRAPHICS:
        marker.undraw()
        del(marker)
        
    if DBG:
        print ('\n',inner,'\n')
    
    #win.getMouse()

print (chain_reactions)
print ()
print (nr_grains)
print()
print (rel_criticality)
print()
print (max_pile)
print()
print (max_pile.max(),max_pile.argmax())
print ()



In [ ]:
(inner >= SOC).sum()
inner.shape[0] * inner.shape[1]

In [ ]:
rel_criticality = pd.Series(rel_criticality)
rel_criticality.plot(figsize=(18,12),style='o--')

In [ ]:
plt.scatter(nr_grains,chain_reactions)

In [ ]:
plt.plot(np.log(nr_grains),np.log(chain_reactions))


In [ ]:
plt.hist(chain_reactions)
plt.yscale('log')
plt.xscale('log')

In [ ]:
chain_reactions = pd.Series(chain_reactions)

freq = chain_reactions.value_counts().sort_index()
freq = freq[1:] ### skip events of 0 chain
print (freq)
freq.plot(logx=True,logy=True,figsize=(18,12),style='o')
plt.xlabel('Avalance Chain Reaction Length')
plt.ylabel('Number of events')

In [ ]:
freq = pd.DataFrame([freq.index.values,freq]).T
freq.columns=['chain_len','count']

#### max chain length ####
_max = freq['chain_len'].max()

#### 
digits_in_max = len( str(int(_max)))
factor = 10 ** (digits_in_max - 1)
uprounded_max = ((_max / factor).astype(int) + 1) * factor

bin_width = factor // 100 if factor >= 1000 else 10

freq['bin'] = pd.cut(freq['chain_len'],
                     range(-bin_width,uprounded_max + bin_width,bin_width),
                     labels=range(0,uprounded_max + bin_width,bin_width))

freq['p'] = freq['count'] / freq['count'].sum()
freq

In [ ]:
freq.plot(y='p',logx=True,logy=True)

In [ ]:
freq.groupby('bin')['p'].sum().plot(logx=True,logy=True)

In [ ]:
freq['bin'] = freq['bin'].astype(int)
freq['count_log'] = np.log(freq['count'])
freq['bin_log'] = np.log(freq['bin'])
freq

In [ ]:
bin_counts = freq.groupby('bin')['count'].sum()
bin_counts.plot(style='o--')

In [ ]:
bin_counts.plot(style='o--',logx=True,logy=True,figsize=(18,12))

In [ ]:
freq.sum()

In [ ]:
freq.describe()

In [ ]:
chain_reactions.plot(figsize=(18,12),style='o--')

In [ ]:
chain_reactions[4000:4200].plot()

In [ ]:
# https://realpython.com/python-scipy-fft/ #
# https://www.softdb.com/what-is-white-noise/ #

from scipy.fft import rfft,rfftfreq,irfft

sample_rate = 1000 # unit of "time" == N : 

signal = chain_reactions
signal = signal - signal.mean()
signal = signal.values

yf = rfft(signal) 
yf = yf / yf.max()

xf = rfftfreq(len(signal),1 / sample_rate)


In [ ]:
yf

In [ ]:
plt.plot(xf,np.abs(yf))
plt.yscale('log')
plt.xscale('log')

In [ ]:
signal = rel_criticality
signal = signal - signal.mean()
signal = signal.values

yf = rfft(signal) 


xf = rfftfreq(len(signal),1 / sample_rate)

In [ ]:
### PINK Noise ###

plt.plot(xf,np.abs(yf))
plt.yscale('log')
plt.xscale('log')

In [ ]:
rel_saturation = pd.Series(rel_saturation)


In [ ]:
signal = rel_saturation.values
signal = signal - signal.mean()
signal = signal

yf = rfft(signal) 
yf = yf / yf.max()

xf = rfftfreq(len(signal),1 / sample_rate)

In [ ]:
plt.plot(xf,np.abs(yf))
plt.yscale('log')
plt.xscale('log')

In [ ]:
if GRAPHICS:
    win.getMouse()
    win.close()



In [ ]:
fig,ax = plt.subplots(figsize=(18,12))

plt.title('Self Organized Criticality - Sandpile model')
ax.plot(chain_reactions,'o--',color='r',label='chain reaction')

ax2 = ax.twinx()
ax2.plot(rel_criticality,'o--',color='g',label='relative system criticality')

ax.set_ylabel('chain reaction length')
ax2.set_ylabel('system relative criticality level')

ax.legend(loc='upper left')
ax2.legend(loc='upper right')

ax.set_xlabel('Iteration')

plt.savefig('SOC_rel_criticality.jpg',format='jpg')

In [ ]:
start = 5400
stop = 5500

plt.figure(figsize=(18,12))

ax = plt.gca()

ax.plot(chain_reactions[start : stop+1],'o--',color='r')

ax2 = plt.twinx()

ax2.plot(rel_criticality[start : stop+1],'o--',color='g')

step = 1
_= ax.set_xticks(range(start,stop+1,step))
_= ax.set_xticklabels(range(start,stop+1,step),rotation=90)


for label in ax.xaxis.get_ticklabels()[::2]:
    label.set_visible(False)
    
ax2.grid(None)

In [ ]:
rel_criticality = np.round(rel_criticality,2)
rel_criticality

In [ ]:
rel_criticality.max()

In [ ]:


rel_crit_binned = pd.cut(rel_criticality,np.arange(rel_criticality.min(),rel_criticality.max() + 0.01,0.01),
                         labels=np.arange(rel_criticality.min(),rel_criticality.max(),0.01))

rel_crit_binned = rel_crit_binned.astype(float)

rel_crit_binned.value_counts().sort_index().plot(figsize=(18,12),style='o--')

plt.ylabel('count')
plt.xlabel('system relative criticality')

In [ ]:
plt.scatter(nr_grains,rel_criticality)

In [ ]:
rel_saturation.plot()

In [ ]:
plt.plot(rel_saturation)
plt.plot(rel_criticality)

In [ ]:
plt.plot(nr_grains,rel_saturation)

In [ ]:
plt.figure(figsize=(18,12))
plt.plot(nr_grains,rel_criticality)

In [ ]:
plt.plot(rel_saturation,rel_criticality)

In [ ]:
plt.plot(rel_saturation,chain_reactions)

In [ ]:
plt.plot(np.log(rel_saturation),np.log(chain_reactions))


In [ ]:
summary_data = pd.concat([chain_reactions,rel_criticality,rel_saturation],axis=1)
summary_data.columns = ['chain_reactions','rel_criticality','rel_saturation']
summary_data = summary_data.sort_values('chain_reactions',ascending=False)
summary_data

In [ ]:
plt.plot(nr_grains[:100000])
# about 10000 iterations onwards, the number of grains is stable #

In [ ]:
# after "steady state" reached, the number of grains seems to be normally distributed
_=plt.hist(nr_grains[20000:],bins=20)

In [ ]:
# chain reactions are not normally dist, but a power law # 
_= plt.hist(chain_reactions[20000:],bins=100)
plt.yscale('log')
plt.xscale('log')